In [1]:
!pip install spacy==3.0.6

  Using cached spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl (12.8 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.1
    Uninstalling spacy-3.2.1:
      Successfully uninstalled spacy-3.2.1


In [2]:
!pip install -U spacy-nightly --pre

In [3]:
!pip install -U pip setuptools wheel

In [4]:
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 459.7 MB 17 kB/s              
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [5]:
!pip install -U spacy transformers

  Using cached spacy-3.2.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Using cached transformers-4.15.0-py3-none-any.whl (3.4 MB)
  Using cached huggingface_hub-0.2.1-py3-none-any.whl (61 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
  Attempting uninstall: transformers
    Found existing installation: transformers 4.9.2
    Uninstalling transformers-4.9.2:
      Successfully uninstalled transformers-4.9.2
  Attempting uninstall: spacy
    Found existing installation: spacy 3.0.6
    Uninstalling spacy-3.0.6:
      Successfully uninstalled spacy-3.0.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.0.4 requires transformers<4.10.0,>=3.4.0, but you have transformers 4.15

In [6]:
!pip install spacy-transformers

  Using cached transformers-4.9.2-py3-none-any.whl (2.6 MB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.2.1
    Uninstalling huggingface-hub-0.2.1:
      Successfully uninstalled huggingface-hub-0.2.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.15.0
    Uninstalling transformers-4.15.0:
      Successfully uninstalled transformers-4.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-trf 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 3.2.1 which is incompatible.


In [7]:
%cd /content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization

/content/drive/MyDrive/TA_project/progetto_TA/NLP_Anonymization


In [8]:
import random

from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
#from spacy.training.example import Example
from rel_pipe import make_relation_extractor, score_relations
from rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [31]:
nlp2 = spacy.load("training/model-best")

In [108]:
text = "con decreto del 12 gennaio 2021 il giudice per le indagini preliminari del tribunale di pavia disponeva il sequestro preventivo dei beni in danno della societ\u00e0 biolevano s.r.l. fino alla concorrenza della somma di \u2022 143.321.004,00, in vista della successiva confisca diretta del reato e, solo qualora la confisca diretta non fosse stata possibile per mancata e/o inferiore provvista, il sequestro preventivo per equivalente dei beni di tutti gli indagati, compreso sguazzini simone, indagato per  ".lower() 
text

'con decreto del 12 gennaio 2021 il giudice per le indagini preliminari del tribunale di pavia disponeva il sequestro preventivo dei beni in danno della società biolevano s.r.l. fino alla concorrenza della somma di • 143.321.004,00, in vista della successiva confisca diretta del reato e, solo qualora la confisca diretta non fosse stata possibile per mancata e/o inferiore provvista, il sequestro preventivo per equivalente dei beni di tutti gli indagati, compreso sguazzini simone, indagato per  '

In [102]:
import spacy.cli

spacy.cli.download('it_core_news_lg')

✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')


In [109]:
nlp_it = spacy.load('it_core_news_lg')
documento = nlp_it(text)

In [110]:
sentenza = nlp2(documento)

In [111]:
probs = 0
imputato = ''
for value, rel_dict in sentenza._.rel.items():
    print(value)
    print(rel_dict)
                    
print(f'imputato: {imputato} con una probabilità di {probs}')                    

(15, 76)
{'DIFENDE': 6.717259e-05, 'GIUDICA': 2.4912217e-07}
(76, 15)
{'DIFENDE': 0.025637487, 'GIUDICA': 0.0030050457}
imputato:  con una probabilità di 0


In [112]:
probs = 0
imputato = ''
for value, rel_dict in sentenza._.rel.items():
    for e in sentenza.ents:
        
        for b in sentenza.ents:
            
            
            if e.start == value[0] and b.start == value[1]:
                if rel_dict['DIFENDE'] > rel_dict['GIUDICA']:
                    if rel_dict['DIFENDE'] > probs:
                        
                        probs = rel_dict['DIFENDE']
                        imputato = b.text
                    
                    
                else:
                    if rel_dict['GIUDICA'] > probs:
                        probs = rel_dict['GIUDICA']
                        imputato = b.text
                    
print(f'imputato: {imputato} con una probabilità di {probs}')                    

imputato: pavia con una probabilità di 0.025637486949563026
